In [55]:
from openai import OpenAI
from tqdm import tqdm
import json
import numpy as np

## 模型准备

In [3]:
# CUDA_VISIBLE_DEVICES=0
# python -m vllm.entrypoints.openai.api_server --served-model-name Qwen2.5-72B-Instruct --model /gemini/data-1/Qwen2.5-72B-Instruct-GPTQ-Int4/ --port 8000 --tensor-parallel-size 2
# python -m vllm.entrypoints.openai.api_server --served-model-name Qwen2.5-7B-Instruct --model /gemini/data-1/Qwen2.5-7B-Instruct/ --port 8000 --tensor-parallel-size 2

In [4]:
# 载入部署好的模型
client = OpenAI(
    api_key="EMPTY",
    base_url="http://localhost:8000/v1",
)   

## 数据准备

In [25]:
# 读取原始数据并处理成需要的格式
def dataset_jsonl_transfer(origin_path, new_path=None):
    """
    将原始数据集转换为大模型微调所需数据格式的新数据集
    """
    messages = []
    Real_outputs = []

    # 读取旧的JSONL文件
    with open(origin_path, "r") as file:
        for line in file:
            # 解析每一行的json数据
            data = json.loads(line)
            input_text = data["text"]
            entities = data["entities"]
            match_names = ["地点", "人名", "地理实体", "组织"]
            
            entity_sentence = ""
            for entity in entities:
                entity_json = dict(entity)
                entity_text = entity_json["entity_text"]
                entity_names = entity_json["entity_names"]
                for name in entity_names:
                    if name in match_names:
                        entity_label = name
                        break
                
                entity_sentence += f"""{{"entity_text": "{entity_text}", "entity_label": "{entity_label}"}}"""
            
            if entity_sentence == "":
                entity_sentence = "没有找到任何实体"
                
            
            message = [
                {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
                {"role": "user", "content": """你是一个文本实体识别领域的专家，你需要从给定的句子中提取 地点; 人名; 地理实体; 组织 实体. 以 json 格式输出, 如 {"entity_text": "南京", "entity_label": "地理实体"} 注意: 1. 输出的每一行都必须是正确的 json 字符串. 2. 找不到任何实体时, 输出"没有找到任何实体". 需要你分析的文本为：""" + input_text}
            ]
            
            messages.append(message)
            Real_outputs.append(entity_sentence)
            
        return messages,Real_outputs

In [30]:
# 获取输入数据
messages,Real_outputs = dataset_jsonl_transfer("../OriginalData/chinese_ner_sft/ccfbdci.jsonl")

## 模型推理

In [37]:
Outputs = []
for message in messages[:10]:
    response = client.chat.completions.create(
        model="Qwen2.5-7B-Instruct",
        messages=message,
        stream=False,
        temperature=0.01,
        max_tokens=3000,
    )
    Outputs.append(response)

In [39]:
Outputs[1].choices[0].message.content

'{"entity_text": "阿美达", "entity_label": "人名"}\n{"entity_text": "基督教传教士", "entity_label": "组织"}'

## 结果对比分析

In [41]:
Prompt1 = """对于一个实体识别任务，我现在将给你大语言模型进行识别后的模型输出，以及该实体识别任务的真实结果，你需要对模型输出的质量进行评价。
## 评价的标准如下：
- 标准A：你需要针对模型输出的实体识别结果，如果该结果存在于真实结果中，请你识别结果中给出的"entity_label"是否准确，准确的个数越多，证明识别记过越好。
- 标准B：你需要统计模型输出的结果中，存在于真实结果的数量，其中模型输出的实体识别结果中，存在于真实结果的比重越大，则说明识别效果越好。
- 标准C：你需要识别模型输出结果和真是结果的差异，其中模型输出结果和真实结果重合度越高，证明识别效果越好。
## 计分标准如下
- 根据我给你的标准A、B、C，每个标准从不好到好分为[1,2,3,4,5,6,7,8,9,10]共10个得分等级，你需要先给出每个标准下的得分。
- 根据每个标准下的得分计算出一个平均分作为你最终的得分输出。
## 输出格式
- 你只需要输出最终的得分即可。
- 你的输出格式为：【最终得分：XX】
# 待分析数据
- 模型输出：{}
- 真实结果：{}
"""

In [53]:
def analysis_outcome(Outputs, Real_outputs):
    # 得分
    scores = []
    # 逐条分析
    for Output, Real_outputs in tqdm(zip(Outputs, Real_outputs)):
        prompt = Prompt1.format(Output.choices[0].message.content, Real_outputs)
        message = [
            {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
        # 获得结果
        response = client.chat.completions.create(
            model="Qwen2.5-7B-Instruct",
            messages=message,
            stream=False,
            temperature=0.01,
            max_tokens=3000,
        )
        # 抽取得分
        score = 0
        try:
            score = int(response.choices[0].message.content.split("【最终得分：")[1].split("】")[0])
        except:
            score = 0
        scores.append(score)
    return scores

In [54]:
scores = analysis_outcome(Outputs, Real_outputs)

10it [00:02,  3.84it/s]


In [56]:
# 统计不合理数据的数量
print(np.mean(scores))

7.0
